## 2. Cut and Rename Root File

In [ ]:
import uproot
import uproot.writing

def ModifyRoot(directory, root_name, tree_name, branch_names, output_name, new_tree_name, new_branch_names):

    input_file = directory + root_name + '.root'
    with uproot.open(input_file) as file:       
        tree = file[tree_name]
        branches = tree.arrays(branch_names, library="np")
        
    output_file = directory + output_name
    counter = 1
    while True:
        if not os.path.exists(f"{output_file}{counter}.root"):
            output_file = f"{output_file}{counter}.root"
            break
        counter = counter + 1

    with uproot.recreate(output_file) as new_file:
        new_file[new_tree_name] = {new_branch_names[0]: branches[branch_names[0]],
                                   new_branch_names[1]: branches[branch_names[1]]}

In [ ]:
directory = 'RESULTS/'

root_name = '80kev_250M'
tree_name = "G4_PCM"
branch_names = ["PositionX", "PositionY"]

output_name = 'output'
new_tree_name = "Photons"
new_branch_names = ["X_axis", "Y_axis"]

ModifyRoot(directory, root_name, tree_name, branch_names, output_name, new_tree_name, new_branch_names)

## 7.2. Eliminate Noise by Fourier Transform

In [ ]:
import numpy as np
from scipy.fft import fft2, fftshift, ifft2
import matplotlib.pyplot as plt
from scipy import signal

def Denoise(htmp_array, hann, alpha, save_as, cortes_transversales):

    image = htmp_array

    fft_image = fft2(image)
    fft_image = fftshift(fft_image)

    rows, cols = image.shape

    hann = False
    if hann == True:
    
        l = rows * alpha
        a = np.hanning(l)
        b = np.hanning(l)

        padding_size = rows - len(a)
        left_padding = padding_size // 2
        right_padding = padding_size - left_padding
        a = np.pad(a, (left_padding, right_padding), mode='constant')

        padding_size = cols - len(b)
        left_padding = padding_size // 2
        right_padding = padding_size - left_padding
        b = np.pad(b, (left_padding, right_padding), mode='constant')

        window = np.outer(a, b)

    else:

        a = signal.windows.tukey(rows, alpha)
        b = signal.windows.tukey(rows, alpha)
        window = np.outer(a, b)

    fft_image_2 = fft_image * (window)
    fft_image = fftshift(fft_image_2)
    fft_image = (ifft2(fft_image))
    fft_image = (np.abs(fft_image))

    if cortes_transversales == True:
        
        plt.figure(figsize = (7, 3))
        plt.subplot(1, 2, 1)
        plt.plot(a)
        plt.title('Window')
        plt.subplot(1, 2, 2)
        plt.plot(np.abs((fft_image_2[:][rows//2])))
        plt.title('F. Transform Slice')

        plt.figure(figsize = (7, 3))
        plt.subplot(1, 2, 1)
        plt.plot(image[:][rows//2])
        plt.title('Original Slice')
        plt.subplot(1, 2, 2)
        plt.plot(np.abs(fft_image[:][rows//2]))
        plt.title('Denoised Slice')

    plt.figure(figsize = (8, 4))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(image, cmap = 'gray')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.title('Filtered Image')
    plt.imshow(fft_image, cmap = 'gray')
    plt.axis('off')
    if save_as != '': plt.savefig('Results/' + save_as + '.png', dpi = 900)
    plt.show()

    return fft_image

# =========================================================================================================================================

hann = False
alpha = 1
array = htmp_array
save_as = ''
graficas = False
fft_image = Denoise(array, hann, alpha, save_as, graficas)